### Muhammad Ammar Masood_Rabeah Zubair_Adeena Bilal
### 17I-0002_17I-0005_17I-0299
### F20-49-D-MeetingScheduler


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Importing Libraries

In [3]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 132kB/s 
     |████████████████████████████████| 6.8MB 52.5MB/s 
     |████████████████████████████████| 983kB 36.0MB/s 
     |████████████████████████████████| 266kB 51.6MB/s 
     |████████████████████████████████| 1.3MB 42.5MB/s 
     |████████████████████████████████| 1.1MB 45.4MB/s 
     |████████████████████████████████| 471kB 49.2MB/s 
     |████████████████████████████████| 2.9MB 45.9MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp36-none-any.whl size=25276312 sha256=bf24bfc13caa6bdb5ea3f0d5fffbdcc3d20ce60dab037cbad5188a50ff4f51ff
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=5ca1fd68f547b25cc273329f5f6f2f5c4b1f8d7b7564af19eb591ba671d8d916
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [4]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import tensorflow as tf
import ktrain
from ktrain import text
from sklearn.model_selection import train_test_split
import re

#### Loading Data

In [5]:
headers = ["Subject", "From", "To", "Date", "Message", "Label"]

In [6]:
data = pd.read_csv("/content/drive/MyDrive/meeting_nonmeeting_dataset - messages.csv", names = headers, dtype=object)
data.head()

,Subject,From,To,Date,Message,Label
0,"""The best of Gmail, wherever you are""",Gmail Team <mail-noreply@google.com>,Rabeah Zubair <i170005@nu.edu.pk>,8/11/2017,"Subject:\nThe best of Gmail, wherever you are\...",0
1,Tips for using your new inbox,Gmail Team <mail-noreply@google.com>,Rabeah Zubair <i170005@nu.edu.pk>,8/11/2017,Subject:\nTips for using your new inbox\nFrom:...,0
2,"""[ CS Academic Office, Islamabad - Announceme...",SLATE<no-reply@nu.edu.pk>,"""CS Academic Office, Islamabad<no-reply@nu.edu.p""",8/11/2017,"Subject:\n[ CS Academic Office, Islamabad - An...",0
3,"""[ Student Affair, Islamabad - Announcement ]...",SLATE<no-reply@nu.edu.pk>,"""Student Affair, Islamabad<no-reply@nu.edu.pk>""",8/15/2017,"Subject:\n[ Student Affair, Islamabad - Announ...",0
4,[ Introduction to Computing CS 2017-03 ISB - New,SLATE<no-reply@nu.edu.pk>,Introduction to Computing CS 2017-03 ISB<no-rep,8/17/2017,Subject:\n[ Introduction to Computing CS 2017-...,0


In [9]:
#Change datatype of Labels to object(str)

convert_dict = {'Label': str} 
data = data.astype(convert_dict) 

In [8]:
data.dtypes

Subject    object
From       object
To         object
Date       object
Message    object
Label      object
dtype: object

###### Drop Unnecessary Columns

In [10]:
drop_labels = ['From', 'To', 'Date']
data.drop(drop_labels, axis = 1, inplace= True)

In [11]:
data.head()

,Subject,Message,Label
0,"""The best of Gmail, wherever you are""","Subject:\nThe best of Gmail, wherever you are\...",0
1,Tips for using your new inbox,Subject:\nTips for using your new inbox\nFrom:...,0
2,"""[ CS Academic Office, Islamabad - Announceme...","Subject:\n[ CS Academic Office, Islamabad - An...",0
3,"""[ Student Affair, Islamabad - Announcement ]...","Subject:\n[ Student Affair, Islamabad - Announ...",0
4,[ Introduction to Computing CS 2017-03 ISB - New,Subject:\n[ Introduction to Computing CS 2017-...,0


##### Using regular expressions to drop out unnecessary headings, etc

In [16]:
patterns = [r'Subject:(\n)*.*(\n)*From:(\n)*.*(\n)*Date:(\n)*.*(\n)*To:(\n)*.*(\n)*',
            r'(\n)*Attachments:(\n)*.*',
            r'(\n)*An announcement has been added in the.*(\n)*Subject:.*(\n)*Group:.*(\n)*Message:.*\n',
            r'(-)+ Forwarded Message (-)+(\n)*From:.*(\n)*Date:.*(\n)*Subject:.*(\n)*To:.*(\n)*(Cc:.*)*(\n)*',
            r'Your email preferences are currently set to.*(\n)*Here.*what happened in.*(\n)*']

for i in range (0,len(data["Message"])):
  for pattern in patterns:
    message = data["Message"][i]
    data["Message"][i] = re.sub(pattern, '', message)


In [18]:
print(data["Message"])

0       The best of Gmail, wherever you are\n\nGoogle\...
1       Tips for using your new inbox\n\nGoogle\n\nHi ...
2       [ CS Academic Office, Islamabad - Announcement...
3       [ Student Affair, Islamabad - Announcement ] F...
4       [ Introduction to Computing CS 2017-03 ISB - N...
                              ...                        
1624    Google logo\nYour account, your data.\nWe've f...
1625    Classroom\nHi I17-0005 Rabeah,\numair arshad p...
1626    Dear Rabeah Zubair !\n\n(Для русскоязычной вер...
1627    Classroom\nHi I17-0005 Rabeah,\nYour work #08 ...
1628    \t\nGoogle\nSign-in attempt was blocked\n\ti17...
Name: Message, Length: 1629, dtype: object


In [19]:
subject_data = data[["Subject","Label"]]
subject_data

,Subject,Label
0,"""The best of Gmail, wherever you are""",0
1,Tips for using your new inbox,0
2,"""[ CS Academic Office, Islamabad - Announceme...",0
3,"""[ Student Affair, Islamabad - Announcement ]...",0
4,[ Introduction to Computing CS 2017-03 ISB - New,0
...,...,...
1624,Your Google data is ready to download,0
1625,"New announcement: """"For Section C and E.Last W...",0
1626,Popular HSE courses and education programms // П,0
1627,"Due tomorrow: """"#08 - FYP-1 Final Presentation...",0


In [20]:
message_data = data[["Message","Label"]]
message_data

,Message,Label
0,"The best of Gmail, wherever you are\n\nGoogle\...",0
1,Tips for using your new inbox\n\nGoogle\n\nHi ...,0
2,"[ CS Academic Office, Islamabad - Announcement...",0
3,"[ Student Affair, Islamabad - Announcement ] F...",0
4,[ Introduction to Computing CS 2017-03 ISB - N...,0
...,...,...
1624,"Google logo\nYour account, your data.\nWe've f...",0
1625,"Classroom\nHi I17-0005 Rabeah,\numair arshad p...",0
1626,Dear Rabeah Zubair !\n\n(Для русскоязычной вер...,0
1627,"Classroom\nHi I17-0005 Rabeah,\nYour work #08 ...",0


In [21]:
#Dividing subject data into train and test

msk = np.random.rand(len(subject_data)) < 0.75

subject_train = subject_data[msk]
subject_test = subject_data[~msk]

In [22]:
#Dividing message data into train and test

msk = np.random.rand(len(message_data)) < 0.75

message_train = message_data[msk]
message_test = message_data[~msk]

##### Subject data frame with headings: (Subject, Label)
##### Label = 0; Non-meeting Email
##### Label = 1; Meeting Email

In [24]:
(X_train, Y_train) , (X_test, Y_test), preprocess = text.texts_from_df(train_df= subject_train, 
                                                                       text_column= 'Subject', 
                                                                       label_columns= 'Label',
                                                                       val_df = subject_test,
                                                                       maxlen = 400,
                                                                       preprocess_mode = 'bert')                                                      

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [25]:
X_train[0].shape

(1243, 400)

#### Traing and Predicting from BERT Model

##### Model for Subject data

In [26]:
model = text.text_classifier(name='bert',
                             train_data = (X_train, Y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [28]:
# Get Learrner 
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, Y_train),
                             val_data = (X_test, Y_test),
                             batch_size = 6)

In [29]:
#learning rate for this model is 2e-5

learner.fit_onecycle(lr = 2e-5, epochs=3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
208/208 [==============================] - 202s 875ms/step - loss: 0.6077 - accuracy: 0.7858 - val_loss: 0.3371 - val_accuracy: 0.8705
Epoch 2/3
208/208 [==============================] - 184s 884ms/step - loss: 0.3092 - accuracy: 0.8806 - val_loss: 0.2884 - val_accuracy: 0.8731
Epoch 3/3
208/208 [==============================] - 183s 881ms/step - loss: 0.2118 - accuracy: 0.9195 - val_loss: 0.2834 - val_accuracy: 0.8756


In [30]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [31]:
data = ['On Sat, Aug 15, 2020 at 4:10 PM Rabeah Zubair <i170005@nu.edu.pk> wrote:\nSir will you be available in the university though? Because we might be visiting university on monday so we could meet you in person instead, if possible?']
for pattern in patterns:
  message = data[0]
  data[0] = re.sub(pattern, '', message)

In [32]:
predictor.predict(data)

['0']

##### Model for Message data

In [33]:
(X1_train, Y1_train) , (X1_test, Y1_test), preprocess1 = text.texts_from_df(train_df= message_train, 
                                                                       text_column= 'Message', 
                                                                       label_columns= 'Label',
                                                                       val_df = message_test,
                                                                       maxlen = 512,
                                                                       preprocess_mode = 'bert')    

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [37]:
model1 = text.text_classifier(name='bert',
                             train_data = (X1_train, Y1_train),
                             preproc = preprocess1)

Is Multi-Label? False
maxlen is 512
done.


In [35]:
# Get Learrner 
learner1 = ktrain.get_learner(model = model1,
                             train_data = (X1_train, Y1_train),
                             val_data = (X1_test, Y1_test),
                             batch_size = 6)

In [36]:
learner1.fit_onecycle(lr = 2e-5, epochs=3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
208/208 [==============================] - 255s 1s/step - loss: 0.7225 - accuracy: 0.6803 - val_loss: 0.3938 - val_accuracy: 0.8497
Epoch 2/3
208/208 [==============================] - 236s 1s/step - loss: 0.3592 - accuracy: 0.8222 - val_loss: 0.2583 - val_accuracy: 0.9067
Epoch 3/3
208/208 [==============================] - 237s 1s/step - loss: 0.1587 - accuracy: 0.9413 - val_loss: 0.2328 - val_accuracy: 0.9249


In [38]:
predictor1 = ktrain.get_predictor(learner1.model, preprocess1)

In [40]:
data = ['Subject:\nNew announcement: "Dear all kindly arrange a meeting with…"\nFrom:\n"Amna Irum (Classroom)" <no-reply+558aa983@classroom.google.com>\nDate:\n21/09/2020, 11:48 am\nTo:i170005@nu.edu.pk\n\nClassroom\nHi Rabeah,\nAmna Irum posted a new announcement in FYP-Fall-2020.\n                \nAnnouncement\n        \nDear all kindly arrange a meeting with on google meet this week for discussion.']

In [41]:
predictor1.predict(data)

['1']

In [42]:
model.save('classify_subject')

In [43]:
model1.save('classify_message')